In [49]:
# Imports
import pyrealsense2 as rs
import numpy as np
import cv2
import os
from pathlib import Path

In [50]:
# input and output paths
bag_file = Path("Sources") / "LapockaVideok" / "20251110_145034.bag"
output_dir = Path("dataset_output") / "session_09"
os.makedirs(output_dir, exist_ok=True)
os.makedirs(output_dir / "rgb", exist_ok=True)
os.makedirs(output_dir / "depth", exist_ok=True)

In [51]:
# RealSense pipeline setup
pipeline = rs.pipeline()
config = rs.config()
config.enable_device_from_file(str(bag_file), repeat_playback=False)
config.enable_stream(rs.stream.color, 848, 480, rs.format.rgb8, 15)
config.enable_stream(rs.stream.depth, 848, 480, rs.format.z16, 15)
profile = pipeline.start(config)

align_to = rs.stream.color
align = rs.align(align_to)


In [52]:
# Video trimming parameters

START_TIME = 3.5
END_TIME = 10.0

# --- playback object for timestamp control ---
device = profile.get_device()
playback = device.as_playback()
playback.set_real_time(False)  # ensures deterministic playback

In [53]:
# Frame capture, alignment and saving (trimmed)
frame_count = 0  # counts only saved frames
start_timestamp = None  # timestamp (s) of first received color frame

print(f"Trimming: keeping frames from {START_TIME:.3f}s to {END_TIME:.3f}s")

try:
    while True:
        frames = pipeline.wait_for_frames()
        # Align depth to color
        aligned_frames = align.process(frames)
        color_frame = aligned_frames.get_color_frame()
        depth_frame = aligned_frames.get_depth_frame()

        if not color_frame or not depth_frame:
            continue

        # Use color frame timestamp for consistent timing (ms -> s)
        ts_sec = color_frame.get_timestamp() / 1000.0
        if start_timestamp is None:
            start_timestamp = ts_sec  # reference start of recording
        rel_time = ts_sec - start_timestamp

        # Skip until we reach START_TIME
        if rel_time < START_TIME:
            continue

        # Stop after END_TIME
        if rel_time > END_TIME:
            break

        # Convert frames to numpy arrays
        color_image = np.asanyarray(color_frame.get_data())
        color_image = cv2.cvtColor(color_image, cv2.COLOR_RGB2BGR) # Convert RGB to BGR for OpenCV
        depth_image = np.asanyarray(depth_frame.get_data())

        # Depth scaling for visualization
        depth_colormap = cv2.applyColorMap(
            cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET
        )

        # Save only trimmed frames (sequentially numbered from 0)
        cv2.imwrite(str(output_dir / "rgb" / f"frame_{frame_count:05d}.png"), color_image)
        np.save(str(output_dir / "depth" / f"frame_{frame_count:05d}.npy"), depth_image)
        cv2.imwrite(str(output_dir / "depth" / f"frame_{frame_count:05d}_colormap.png"), depth_colormap)

        frame_count += 1

except RuntimeError:
    print("Reached end of .bag file or stream error")

pipeline.stop()
print(f"Exported {frame_count} trimmed frames between {START_TIME}s and {END_TIME}s (relative)")

Trimming: keeping frames from 3.500s to 10.000s
Exported 97 trimmed frames between 3.5s and 10.0s (relative)


In [54]:
# Cropping region of interest and saving

cropped_img_path = output_dir / "cropped"
os.makedirs(cropped_img_path / "rgb", exist_ok=True)
os.makedirs(cropped_img_path / "depth", exist_ok=True)

roi = (310, 43, 320, 320)  # (x, y, w, h)
for i in range(frame_count):
    rgb = cv2.imread(str(output_dir / "rgb" / f"frame_{i:05d}.png"))
    depth = np.load(str(output_dir / "depth" / f"frame_{i:05d}.npy"))
    depth_colormap = cv2.imread(str(output_dir / "depth" / f"frame_{i:05d}_colormap.png"))

    rgb_crop = rgb[roi[1]:roi[1]+roi[3], roi[0]:roi[0]+roi[2]]
    depth_crop = depth[roi[1]:roi[1]+roi[3], roi[0]:roi[0]+roi[2]]
    depth_colormap_crop = depth_colormap[roi[1]:roi[1]+roi[3], roi[0]:roi[0]+roi[2]]

    cv2.imwrite(str(cropped_img_path / "rgb" / f"frame_{i:05d}_crop.png"), rgb_crop)
    np.save(str(cropped_img_path / "depth" / f"frame_{i:05d}_crop.npy"), depth_crop)
    cv2.imwrite(str(cropped_img_path / "depth" / f"frame_{i:05d}_colormap_crop.png"), depth_colormap_crop)